In [29]:
from google.colab import drive

# Google Drive 연결
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# 필요한 라이브러리 설치
!pip install transformers datasets gensim nltk -q

In [19]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import re
from transformers import RobertaTokenizer, RobertaForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu
import torch
from gensim.models import KeyedVectors

In [20]:
# 데이터 전처리 함수 정의
def preprocess_text(text):
    text = re.sub(r"http\S+", "", text)  # URL 제거
    text = re.sub(r"@\w+", "", text)    # 사용자 태그 제거
    text = re.sub(r"[^a-zA-Z]", " ", text)  # 특수 문자 제거
    text = text.lower()  # 소문자로 변환
    return text.strip()

# 데이터 로드
url = "https://raw.githubusercontent.com/t-davidson/hate-speech-and-offensive-language/master/data/labeled_data.csv"
data = pd.read_csv(url)

# 데이터 전처리
data['clean_text'] = data['tweet'].apply(preprocess_text)

# 데이터셋 분리
train_texts, val_texts, train_labels, val_labels = train_test_split(
    data['clean_text'], data['class'], test_size=0.2, random_state=42
)

# 토크나이저 및 데이터 토큰화
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True, max_length=128)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True, max_length=128)

# 데이터셋 클래스 정의
class HateSpeechDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels.iloc[idx], dtype=torch.long)
        return item

train_dataset = HateSpeechDataset(train_encodings, train_labels)
val_dataset = HateSpeechDataset(val_encodings, val_labels)

In [21]:
# 모델 로드
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)

# Trainer 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,  # 에포크를 20으로 설정
    weight_decay=0.01,
    report_to="none"
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
import matplotlib.pyplot as plt
from transformers import TrainerCallback

# 에폭별 결과를 저장하는 콜백 클래스 정의
class MetricsCallback(TrainerCallback):
    def __init__(self):
        self.metrics = {"epoch": [], "accuracy": [], "loss": [], "f1": []}

    def on_epoch_end(self, args, state, control, **kwargs):
        # 에폭별 측정값 저장
        logs = state.log_history[-1]  # 가장 최근 로그
        self.metrics["epoch"].append(state.epoch)
        self.metrics["accuracy"].append(logs.get("eval_accuracy", None))
        self.metrics["loss"].append(logs.get("eval_loss", None))
        self.metrics["f1"].append(logs.get("eval_f1", None))

# 콜백 객체 생성
metrics_callback = MetricsCallback()

In [23]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average="weighted")
    return {"accuracy": acc, "f1": f1}

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[metrics_callback],
)

In [24]:
# 모델 학습
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.280100,0.262963,0.910026,0.906977
2,0.237700,0.272307,0.912851,0.903976
3,0.190400,0.308223,0.907605,0.905860


TrainOutput(global_step=3720, training_loss=0.25019106095837007, metrics={'train_runtime': 766.5834, 'train_samples_per_second': 77.588, 'train_steps_per_second': 4.853, 'total_flos': 3912364964906496.0, 'train_loss': 0.25019106095837007, 'epoch': 3.0})

In [25]:
# GloVe 모델 다운로드
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip -q glove.6B.zip -d glove

--2024-12-09 09:04:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-12-09 09:04:27--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-12-09 09:04:27--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.1’

gl

In [26]:
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import KeyedVectors
import torch

# GloVe 파일 경로
glove_file = "glove/glove.6B.100d.txt"
word2vec_output_file = "glove.6B.100d.word2vec.txt"

# GloVe 형식을 Word2Vec 형식으로 변환 및 로드
try:
    glove2word2vec(glove_file, word2vec_output_file)
    glove_model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)
    print("GloVe 모델 로드 성공!")
except Exception as e:
    print(f"GloVe 모델 로드 실패: {e}")
    raise

# 반복 검증을 위한 함수 정의
def verify_and_neutralize(sentence, model, tokenizer, glove_model, max_attempts=5, epsilon=1.0, noise_scale=0.1):
    device = next(model.parameters()).device
    harmful_words = {"stupid", "idiot", "hate", "terrible", "dumb"}  # 확장된 유해 단어 리스트
    for attempt in range(max_attempts):
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
        model.eval()
        with torch.no_grad():
            logits = model(**inputs).logits
        predicted_label = logits.argmax(axis=-1).item()

        if predicted_label == 0:
            print(f"Attempt {attempt + 1}: Sentence is classified as neutral.")
            return sentence

        tokens = tokenizer.tokenize(sentence)
        neutralized_tokens = []
        for token in tokens:
            if token in harmful_words and token in glove_model:
                similar_words = glove_model.most_similar(positive=[token], topn=1)
                replacement = similar_words[0][0]
                print(f"Detected token: {token}, Similar words: {similar_words}")
                neutralized_tokens.append(replacement)
            else:
                neutralized_tokens.append(token)

        for i, token in enumerate(neutralized_tokens):
            if token in glove_model:
                vector = glove_model[token].copy()
                noise = torch.distributions.Laplace(0, noise_scale).sample(torch.tensor(vector).size())
                print(f"Original vector: {glove_model[token]}, Noised vector: {vector}")
                vector += noise.cpu().numpy()
                neutralized_tokens[i] = glove_model.most_similar(positive=[vector], topn=1)[0][0]

        sentence = tokenizer.convert_tokens_to_string(neutralized_tokens)
        print(f"Attempt {attempt + 1}: Neutralized Sentence: {sentence}")

    print("Max attempts reached. Sentence remains harmful. Applying censorship.")
    return "[CENSORED]"


<ipython-input-26-3c7092146f49>:11: DeprecationWarning: Call to deprecated `glove2word2vec` (KeyedVectors.load_word2vec_format(.., binary=False, no_header=True) loads GLoVE text vectors.).
  glove2word2vec(glove_file, word2vec_output_file)


GloVe 모델 로드 성공!


In [37]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 평가 함수 수정
def evaluate_model(test_sentences, model, tokenizer, glove_model, max_attempts=5):
    # Device 설정
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    original_labels, predicted_labels = [], []
    success_count = 0

    for sentence in test_sentences:
        # 입력 데이터를 디바이스로 이동
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        original_label = logits.argmax(axis=-1).item()
        original_labels.append(original_label)

        # 중립화 실행
        neutralized_sentence = verify_and_neutralize(sentence, model, tokenizer, glove_model, max_attempts)
        inputs = tokenizer(neutralized_sentence, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        predicted_label = logits.argmax(axis=-1).item()
        predicted_labels.append(predicted_label)

        if predicted_label == 0:
            success_count += 1

    # 평가 지표 계산
    precision = precision_score(original_labels, predicted_labels, average="weighted")
    recall = recall_score(original_labels, predicted_labels, average="weighted")
    f1 = f1_score(original_labels, predicted_labels, average="weighted")

    # 결과 출력
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")


In [38]:
import pandas as pd
import re
import torch
from transformers import RobertaTokenizer

# 데이터 전처리 함수 정의
def preprocess_text(text):
    text = re.sub(r"http\S+", "", text)  # URL 제거
    text = re.sub(r"@\w+", "", text)    # 사용자 태그 제거
    text = re.sub(r"[^a-zA-Z]", " ", text)  # 특수 문자 제거
    text = text.lower()  # 소문자로 변환
    return text.strip()

# 데이터 로드
file_path = "/content/drive/MyDrive/Colab Notebooks/조현수/팀플실험/Dataset/test_subset_100.csv"
data = pd.read_csv(file_path)

# `comment_text` 컬럼의 값 전처리
data['clean_text'] = data['comment_text'].apply(preprocess_text)

# 전처리된 텍스트를 리스트로 저장
test_sentences = data['clean_text'].tolist()
print(f"First 5 processed sentences: {test_sentences[:5]}")

# 토크나이저 로드
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# 테스트 데이터 토큰화
test_encodings = tokenizer(test_sentences, truncation=True, padding=True, max_length=128)

# 데이터셋 클래스 정의
class TestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __len__(self):
        return len(self.encodings["input_ids"])

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

# 테스트 데이터셋 생성
test_dataset = TestDataset(test_encodings)

# 테스트 데이터셋 확인
print(f"Test dataset length: {len(test_dataset)}")


First 5 processed sentences: ['yo bitch ja rule is more succesful then you ll ever be whats up with you and hating you sad mofuckas   i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me  ja rule is about pride in da music man  dont diss that shit on him  and nothin is wrong bein like tupac he was a brother too   fuckin white boys get things right next time', 'from rfc       the title is fine as it is  imo', 'sources         zawe ashton on lapland', 'if you have a look back at the source  the information i updated was the correct form  i can only guess the source hadn t updated  i shall update the information once again but thank you for your message', 'i don t anonymously edit articles at all']
Test dataset length: 100


In [39]:

# 함수 호출
evaluate_model(test_sentences, model, tokenizer, glove_model)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  0.4652    0.66263  -0.72842  -0.53341  -0.146    -0.49811   0.18418
  0.069905  0.55154  -0.29809   0.33195  -0.12291   0.73052  -0.39608
  0.090747 -0.20109  -0.68028   0.96415   0.15984   1.377    -0.63242
  0.19293  -0.39268   0.30037   0.54958  -0.19504   0.36234  -0.709
 -0.22919  -0.63709 ], Noised vector: [-0.126    -0.3407    0.10885  -0.2068   -0.34073  -0.70748   0.27933
 -0.24642  -0.29863  -0.037723 -0.11446   0.59435   0.42439  -0.24444
  0.55089  -0.21384   0.14017   0.87152  -0.27218   0.19693  -0.20669
  0.89598  -0.353     0.23329  -0.31833   0.4098   -0.79132  -0.71471
 -0.33074   0.091787  0.82255  -0.42403  -0.69841  -0.6281   -0.30795
 -0.99922  -0.24437   0.72319  -0.084056 -1.1771    0.31028   0.085751
  0.35332   0.13038   0.16771  -0.21867  -1.6139    0.38261   0.85699
  0.085725 -0.20913   0.23075  -0.14548  -0.11903  -0.92269   0.81827
  0.68425   0.48242  -0.35566  -0.95499   0.0709   -0.2311    0.24814
 -1.194    -0.154

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


###아예 따로 만들었음

In [46]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu
import numpy as np
import pandas as pd
from transformers import pipeline
import torch.distributions as dist

# Zero-shot 결과 평가 함수
def evaluate_zero_shot(test_sentences, zero_shot_classifier):
    labels = ["Toxic", "Neutral"]  # Zero-shot 평가에 사용할 레이블
    original_labels, predicted_labels = [], []
    cosine_scores, bleu_scores = [], []

    for sentence in test_sentences:
        try:
            # Zero-shot 분류 실행
            result = zero_shot_classifier(sentence, labels, hypothesis_template="This text is {}.")
            predicted_label = result["labels"][0]
            score = result["scores"][0]

            # Toxic: 1, Neutral: 0으로 매핑
            predicted_labels.append(1 if predicted_label == "Toxic" else 0)

            # Cosine similarity 및 BLEU Score를 Zero-shot에서 계산 (생략)
            cosine_scores.append(0)
            bleu_scores.append(0)
        except Exception as e:
            print(f"Error during Zero-shot classification: {e}")
            predicted_labels.append(0)
            cosine_scores.append(0)
            bleu_scores.append(0)

    # 평가 지표 계산
    true_labels = [1 if "toxic" in s.lower() else 0 for s in test_sentences]  # 간단한 기준으로 레이블 생성
    accuracy = accuracy_score(true_labels, predicted_labels)
    precision = precision_score(true_labels, predicted_labels, average="weighted")
    recall = recall_score(true_labels, predicted_labels, average="weighted")
    f1 = f1_score(true_labels, predicted_labels, average="weighted")

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "Cosine similarity": np.mean(cosine_scores),
        "BLEU scores": np.mean(bleu_scores)
    }

# Fine-tuned 모델 평가 함수
def evaluate_and_store_results(test_sentences, model, tokenizer, glove_model, max_attempts=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    original_labels, predicted_labels = [], []
    cosine_scores, bleu_scores = [], []

    for sentence in test_sentences:
        # 원본 문장의 유해성 분류
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        original_label = logits.argmax(axis=-1).item()
        original_labels.append(original_label)

        # 중립화 실행
        neutralized_sentence = verify_and_neutralize(sentence, model, tokenizer, glove_model, max_attempts)
        inputs = tokenizer(neutralized_sentence, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
        predicted_label = logits.argmax(axis=-1).item()
        predicted_labels.append(predicted_label)

        # 유사도 계산
        original_vectors = [glove_model[token] for token in tokenizer.tokenize(sentence) if token in glove_model]
        neutralized_vectors = [glove_model[token] for token in tokenizer.tokenize(neutralized_sentence) if token in glove_model]
        if original_vectors and neutralized_vectors:
            cosine_sim = cosine_similarity(
                np.mean(original_vectors, axis=0).reshape(1, -1),
                np.mean(neutralized_vectors, axis=0).reshape(1, -1)
            )[0, 0]
            cosine_scores.append(cosine_sim)

        # BLEU Score 계산
        bleu_scores.append(sentence_bleu([sentence.split()], neutralized_sentence.split(), weights=(0.5, 0.5)))

    # 평가 지표 계산
    accuracy = accuracy_score(original_labels, predicted_labels)
    precision = precision_score(original_labels, predicted_labels, average="weighted")
    recall = recall_score(original_labels, predicted_labels, average="weighted")
    f1 = f1_score(original_labels, predicted_labels, average="weighted")

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "Cosine similarity": np.mean(cosine_scores) if cosine_scores else 0,
        "BLEU scores": np.mean(bleu_scores) if bleu_scores else 0
    }

# Proposed method with DP
# Proposed method with DP
def evaluate_proposed_method_with_dp(test_sentences, model, tokenizer, glove_model, max_attempts=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    original_labels, predicted_labels = [], []
    cosine_scores, bleu_scores = [], []

    for sentence in test_sentences:
        # Original label with DP
        inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            # Add Differential Privacy noise
            noise = dist.Normal(0, 0.1).sample(logits.size()).to(device)
            logits += noise  # DP 적용
        original_label = logits.argmax(axis=-1).item()
        original_labels.append(original_label)

        # Neutralize sentence
        neutralized_sentence = verify_and_neutralize(sentence, model, tokenizer, glove_model, max_attempts)
        inputs = tokenizer(neutralized_sentence, return_tensors="pt", truncation=True, padding=True).to(device)
        with torch.no_grad():
            logits = model(**inputs).logits
            logits += noise  # DP noise 재적용
        predicted_label = logits.argmax(axis=-1).item()
        predicted_labels.append(predicted_label)

        # Cosine similarity and BLEU scores
        original_vectors = [glove_model[token] for token in tokenizer.tokenize(sentence) if token in glove_model]
        neutralized_vectors = [glove_model[token] for token in tokenizer.tokenize(neutralized_sentence) if token in glove_model]
        if original_vectors and neutralized_vectors:
            cosine_sim = cosine_similarity(
                np.mean(original_vectors, axis=0).reshape(1, -1),
                np.mean(neutralized_vectors, axis=0).reshape(1, -1)
            )[0, 0]
            cosine_scores.append(cosine_sim)

        bleu_scores.append(sentence_bleu([sentence.split()], neutralized_sentence.split(), weights=(0.5, 0.5)))

    # Compute metrics
    accuracy = accuracy_score(original_labels, predicted_labels)
    precision = precision_score(original_labels, predicted_labels, average="weighted")
    recall = recall_score(original_labels, predicted_labels, average="weighted")
    f1 = f1_score(original_labels, predicted_labels, average="weighted")

    return {
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1-score": f1,
        "Cosine similarity": np.mean(cosine_scores) if cosine_scores else 0,
        "BLEU scores": np.mean(bleu_scores) if bleu_scores else 0
    }

# Result storage function
def store_results_in_table(results, method_name):
    table = {
        "Method": [method_name],
        "Classification performance: Accuracy": [results["Accuracy"]],
        "Classification performance: Precision": [results["Precision"]],
        "Classification performance: Recall": [results["Recall"]],
        "Classification performance: F1-score": [results["F1-score"]],
        "Semantic preservation: Cosine similarity": [results["Cosine similarity"]],
        "Semantic preservation: BLEU scores": [results["BLEU scores"]]
    }
    return pd.DataFrame(table)

# Fine-tuned 모델 평가
results_finetuned = evaluate_and_store_results(test_sentences, model, tokenizer, glove_model)
table_finetuned = store_results_in_table(results_finetuned, "RoBERTa Fine-tuned")

# Zero-shot 평가
results_zeroshot = evaluate_zero_shot(test_sentences, zero_shot_classifier)
table_zeroshot = store_results_in_table(results_zeroshot, "Zero-shot* (facebook/bart-large-mnli)")

# Proposed method with DP 평가
results_proposed = evaluate_proposed_method_with_dp(test_sentences, model, tokenizer, glove_model)
table_proposed = store_results_in_table(results_proposed, "Proposed method** (RoBERTa Fine-tuned + DP)")

# 최종 표 결합
final_table = pd.concat([table_finetuned, table_zeroshot, table_proposed])

# 저장 및 출력
final_table.to_csv("classification_performance_and_semantic_preservation.csv", index=False)
print(final_table)



스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
 -1.194    -0.15417  -0.31809   0.24604  -0.20091  -0.86818  -0.22694
  0.4652    0.66263  -0.72842  -0.53341  -0.146    -0.49811   0.18418
  0.069905  0.55154  -0.29809   0.33195  -0.12291   0.73052  -0.39608
  0.090747 -0.20109  -0.68028   0.96415   0.15984   1.377    -0.63242
  0.19293  -0.39268   0.30037   0.54958  -0.19504   0.36234  -0.709
 -0.22919  -0.63709 ], Noised vector: [-0.126    -0.3407    0.10885  -0.2068   -0.34073  -0.70748   0.27933
 -0.24642  -0.29863  -0.037723 -0.11446   0.59435   0.42439  -0.24444
  0.55089  -0.21384   0.14017   0.87152  -0.27218   0.19693  -0.20669
  0.89598  -0.353     0.23329  -0.31833   0.4098   -0.79132  -0.71471
 -0.33074   0.091787  0.82255  -0.42403  -0.69841  -0.6281   -0.30795
 -0.99922  -0.24437   0.72319  -0.084056 -1.1771    0.31028   0.085751
  0.35332   0.13038   0.16771  -0.21867  -1.6139    0.38261   0.85699
  0.085725 -0.20913   0.23075  -0.14548  -0.11903  -0.92269   0.81827
  0.68425   0.482

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Error during Zero-shot classification: You must include at least one label and at least one sequence.


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  0.38549   0.61993  -1.032     0.70119  -0.2246    0.079435  0.09126
 -0.21196  -0.55429  -0.053352 -0.80201   0.46798  -0.05005  -0.57422
 -0.084822 -1.7227   -0.94286   0.98667   0.31211  -0.37735   0.068674
 -0.77838  -0.28486   0.81047   0.46596  -0.11865  -0.93411   0.33722
  0.037906 -0.18273  -0.019941  0.20494  -0.47718  -0.49253  -0.56518
  0.72558  -0.15913 ], Noised vector: [-0.4214   -0.18797   0.46241  -0.17605   0.36212   0.36701   0.27924
  0.14634  -0.054227  0.45834   0.065416 -0.33725   0.067505 -0.36316
  0.50302  -0.010361  0.72826  -0.17564  -0.33996   0.072864  0.64481
 -0.23908   0.38383   0.13858   1.0994   -0.24883  -0.15078  -0.48738
 -0.23042   0.064788 -0.70183   0.82654   0.06128   0.18531  -0.30162
 -0.022151  0.34302   0.80331   0.17135   0.15462  -0.50759   0.39572
  0.054291 -0.53081   0.48252   0.086205  0.59585  -0.22377  -0.3955
 -0.73036  -0.10279  -0.39166   1.229     1.2129   -1.0365   -3.4971
  0.10923  -1.008

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
